# NLP

## Setup 

This setup allows you to use *Python* and *R* in the same notebook.


In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


In [4]:
from tqdm.notebook import tqdm
tqdm.pandas()

## Load Data & Remove Duplciates 🧹

In [5]:
bills = pd.read_csv('bill_search.csv')
bills = bills[0:-1]
bills

,State,Bill Number,Name,Summary,Bill Progress,Last Action,Action Date
0,MD,HB1,Maryland Paint Stewardship,Requiring producers of architectural paint or ...,Crossed Over,"Referred Education, Energy, and the Environment",02/26/2024
1,MD,HB2,Baltimore City - Property Taxes - Authority to...,AN ACT concerning Baltimore City - Property Ta...,In Committee,House Ways and Means Hearing (13:00:00 1/25/20...,01/25/2024
2,MD,HB3,Land Use - Expedited Development Review Proces...,AN ACT concerning Land Use - Expedited Develop...,In Committee,House Environment and Transportation Hearing (...,01/30/2024
3,MD,HB6,Public Safety - Law Enforcement - Quotas (Comm...,AN ACT concerning Public Safety - Law Enforcem...,In Committee,House Judiciary Hearing (13:00:00 1/23/2024 ),01/23/2024
4,MD,HB8,Maryland Police Training and Standards Commiss...,AN ACT concerning Maryland Police Training and...,In Committee,House Judiciary Hearing (13:00:00 1/23/2024 ),01/23/2024
...,...,...,...,...,...,...,...
995,MD,SB671,Foreclosure Proceedings - Residential Mortgago...,Requiring that individuals have access to lega...,In Committee,Senate Judicial Proceedings Hearing (13:00:00 ...,02/20/2024
996,MD,SB676,Tax Assistance for Low-Income Marylanders - Fu...,"Requiring the Comptroller, beginning in fiscal...",In Committee,Senate Budget and Taxation Hearing (13:00:00 2...,02/14/2024
997,MD,SB677,Comptroller - Electronic Tax and Fee Return Fi...,"Requiring, beginning in calendar year 2026, th...",In Committee,Senate Budget and Taxation Hearing (13:00:00 2...,02/14/2024
998,MD,SB678,Income Tax - Technical Corrections,Repealing certain obsolete provisions of law c...,Crossed Over,Referred Ways and Means,03/01/2024


duplicates to delete

# Keywords

In [6]:
from yake import KeywordExtractor
from pandarallel import pandarallel

kw_extractor = KeywordExtractor()

def get_keywords(text):
    keywords = kw_extractor.extract_keywords(text)
    return [x for x,y in keywords]

pandarallel.initialize(progress_bar=True)
bills['keywords'] = bills['Summary'].parallel_apply(get_keywords)

# display
bills

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


,State,Bill Number,Name,Summary,Bill Progress,Last Action,Action Date,keywords
0,MD,HB1,Maryland Paint Stewardship,Requiring producers of architectural paint or ...,Crossed Over,"Referred Education, Energy, and the Environment",02/26/2024,"[Paint Stewardship Program, paint beginning Ja..."
1,MD,HB2,Baltimore City - Property Taxes - Authority to...,AN ACT concerning Baltimore City - Property Ta...,In Committee,House Ways and Means Hearing (13:00:00 1/25/20...,01/25/2024,"[special property tax, property tax rate, Vaca..."
2,MD,HB3,Land Use - Expedited Development Review Proces...,AN ACT concerning Land Use - Expedited Develop...,In Committee,House Environment and Transportation Hearing (...,01/30/2024,"[Expedited Development Review, Development Rev..."
3,MD,HB6,Public Safety - Law Enforcement - Quotas (Comm...,AN ACT concerning Public Safety - Law Enforcem...,In Committee,House Judiciary Hearing (13:00:00 1/23/2024 ),01/23/2024,"[law enforcement quotas, law enforcement offic..."
4,MD,HB8,Maryland Police Training and Standards Commiss...,AN ACT concerning Maryland Police Training and...,In Committee,House Judiciary Hearing (13:00:00 1/23/2024 ),01/23/2024,"[United States armed, Police Officer Certifica..."
...,...,...,...,...,...,...,...,...
995,MD,SB671,Foreclosure Proceedings - Residential Mortgago...,Requiring that individuals have access to lega...,In Committee,Senate Judicial Proceedings Hearing (13:00:00 ...,02/20/2024,"[Foreclosure Proceedings Program, Maryland Leg..."
996,MD,SB676,Tax Assistance for Low-Income Marylanders - Fu...,"Requiring the Comptroller, beginning in fiscal...",In Committee,Senate Budget and Taxation Hearing (13:00:00 2...,02/14/2024,"[Low-Income Marylanders Fund, mobile tax clini..."
997,MD,SB677,Comptroller - Electronic Tax and Fee Return Fi...,"Requiring, beginning in calendar year 2026, th...",In Committee,Senate Budget and Taxation Hearing (13:00:00 2...,02/14/2024,"[Comptroller be filed, beginning in calendar, ..."
998,MD,SB678,Income Tax - Technical Corrections,Repealing certain obsolete provisions of law c...,Crossed Over,Referred Ways and Means,03/01/2024,"[income tax revenue, Repealing certain obsolet..."


## Embeddings

In [7]:
import os
import openai
import dotenv
dotenv.load_dotenv()

openai.organization = None
openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.Model.list() # see all openai models

In [9]:
# https://github.com/openai/openai-cookbook/blob/main/examples/Obtain_dataset.ipynb

# imports
import pandas as pd
import tiktoken

# from openai.embeddings_utils import get_embedding

from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

# load & inspect dataset
bills["combined"] = (
    "Title: " + bills.Name.str.strip() + "; Content: " + bills.Summary.str.strip()
)
bills['ada_embedding'] = bills.combined.progress_apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
bills.to_csv('output/embedded_1k_reviews.csv', index=False)

bills.head(2)

  0%|          | 0/1000 [00:00<?, ?it/s]

,State,Bill Number,Name,Summary,Bill Progress,Last Action,Action Date,keywords,combined,ada_embedding
0,MD,HB1,Maryland Paint Stewardship,Requiring producers of architectural paint or ...,Crossed Over,"Referred Education, Energy, and the Environment",02/26/2024,"[Paint Stewardship Program, paint beginning Ja...",Title: Maryland Paint Stewardship; Content: Re...,"[0.034104276448488235, 0.0012460323050618172, ..."
1,MD,HB2,Baltimore City - Property Taxes - Authority to...,AN ACT concerning Baltimore City - Property Ta...,In Committee,House Ways and Means Hearing (13:00:00 1/25/20...,01/25/2024,"[special property tax, property tax rate, Vaca...",Title: Baltimore City - Property Taxes - Autho...,"[0.01748131774365902, 0.05112225562334061, 0.0..."


In [ ]:
# encoding = tiktoken.get_encoding(embedding_encoding)

# # omit reviews that are too long to embed
# bills["n_tokens"] = bills.combined.apply(lambda x: len(encoding.encode(x)))
# bills = bills.sort_values(by='n_tokens', ascending=False)
# bills


remove stories that are too long

In [ ]:
# too_long = bills.query("n_tokens > @max_tokens")
# len(f"Removing {len(too_long)} bills that are too long")
# too_long.to_csv('output/too_long.csv', index=False)
# bills = bills.query("n_tokens <= @max_tokens") # remove stories that are too long
# too_long[['title','publication_date','domain','n_tokens']]

In [12]:
bills["embedding"] = bills['ada_embedding']

In [13]:
bills.to_csv('bills-with-embeddings.csv')

## Dimensionality Reduction (t-SNE)


In [14]:
from sklearn.manifold import TSNE
import numpy as np

# check if vis_dims exists
if os.path.exists("data/bills-with-vis-dims.csv"):
    bills = pd.read_csv("data/bills-with-vis-dims.csv")
else: 
    # Convert to a list of lists of floats
    matrix = np.array(bills.embedding.to_list())

    # Create a t-SNE model and transform the data
    tsne = TSNE(n_components=2, perplexity=30, random_state=42, init='random', learning_rate=400)
    vis_dims = tsne.fit_transform(matrix)

    # add to dataframe and write to csv
    bills = bills\
        .assign(
            x = vis_dims[:,0], 
            y = vis_dims[:,1])


In [15]:
bills.to_csv('output/bills-with-nlp.csv', index=False)
bills.head()

,State,Bill Number,Name,Summary,Bill Progress,Last Action,Action Date,keywords,combined,ada_embedding,embedding,x,y
0,MD,HB1,Maryland Paint Stewardship,Requiring producers of architectural paint or ...,Crossed Over,"Referred Education, Energy, and the Environment",02/26/2024,"[Paint Stewardship Program, paint beginning Ja...",Title: Maryland Paint Stewardship; Content: Re...,"[0.034104276448488235, 0.0012460323050618172, ...","[0.034104276448488235, 0.0012460323050618172, ...",15.825431,1.221016
1,MD,HB2,Baltimore City - Property Taxes - Authority to...,AN ACT concerning Baltimore City - Property Ta...,In Committee,House Ways and Means Hearing (13:00:00 1/25/20...,01/25/2024,"[special property tax, property tax rate, Vaca...",Title: Baltimore City - Property Taxes - Autho...,"[0.01748131774365902, 0.05112225562334061, 0.0...","[0.01748131774365902, 0.05112225562334061, 0.0...",41.534931,6.329552
2,MD,HB3,Land Use - Expedited Development Review Proces...,AN ACT concerning Land Use - Expedited Develop...,In Committee,House Environment and Transportation Hearing (...,01/30/2024,"[Expedited Development Review, Development Rev...",Title: Land Use - Expedited Development Review...,"[-0.0070806401781737804, 0.055319491773843765,...","[-0.0070806401781737804, 0.055319491773843765,...",24.153841,-4.798958
3,MD,HB6,Public Safety - Law Enforcement - Quotas (Comm...,AN ACT concerning Public Safety - Law Enforcem...,In Committee,House Judiciary Hearing (13:00:00 1/23/2024 ),01/23/2024,"[law enforcement quotas, law enforcement offic...",Title: Public Safety - Law Enforcement - Quota...,"[0.05756470561027527, 0.04954656586050987, 0.0...","[0.05756470561027527, 0.04954656586050987, 0.0...",-18.685030,-1.792469
4,MD,HB8,Maryland Police Training and Standards Commiss...,AN ACT concerning Maryland Police Training and...,In Committee,House Judiciary Hearing (13:00:00 1/23/2024 ),01/23/2024,"[United States armed, Police Officer Certifica...",Title: Maryland Police Training and Standards ...,"[0.017577573657035828, 0.03067043609917164, 0....","[0.017577573657035828, 0.03067043609917164, 0....",-16.251020,-1.486338


# Topic Modeling

In [ ]:
stories.reset_index(drop=True, inplace=True)

In [ ]:
stories

In [ ]:
from sklearn.cluster import DBSCAN
# Convert embedding to a NumPy array
X = np.stack(stories['embedding'].values)

# Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=10)  # Adjust eps and min_samples as per your requirement
labels = dbscan.fit_predict(X)

# Assign topics to DataFrame
stories['topic'] = labels

# Group articles by topic
grouped = stories.groupby('topic')

# sort groups by size
grouped = sorted(grouped, key=lambda x: len(x[1]), reverse=True)

# assign group numbers back to stories
for i, (name, group) in enumerate(grouped):
    # TODO: I THINK THIS IS BROKEN 🐛, getting weird items into
    stories.loc[stories['topic'] == name, 'topic'] = name

print("Number of groups:", len(grouped))
# Number of items in each group
print("Group sizes:")
print([len(group) for name, group in grouped])



In [ ]:

def summarize_topic(titles):
    """
    Pass list of titles to ChatGPT and ask it to summarize them in 2-4 words.
    """
    # Combine the titles into a single string
    titles_str = ', '.join(titles)

    response = openai.Completion.create(
      engine="text-davinci-002",
      prompt=f"The following article titles form a topic. \n\n {titles_str} \n\n Please write a specific summary of the topic in 2-4 words:",
      max_tokens=10  # Limit the response length
    )

    summary = response.choices[0].text.strip()
    return summary

In [ ]:
# make a list of titles per topic
topic_titles = df.groupby('topic')['title'].apply(list).to_dict()
topic_titles = [{
    'topic': k,
    'num_articles': len(v),
    'headlines': v
} for k,v in topic_titles.items()]

# sort by num_articles
topic_titles = sorted(topic_titles, key=lambda x: x['num_articles'], reverse=True)

# pass each topic list of titles to openai chatgpt and ask it to summarize the topic in 2-4 words
for topic in topic_titles[0:]:
    print(f"Topic {topic['topic']} ({topic['num_articles']} articles)")
    
    if topic['topic'] == -1:
        continue

    # if there are more than 10 articles in a topic, sample 10 (to keep within the word limit of the API)
    if topic['num_articles'] > 10:
        headlines = np.random.choice(topic['headlines'], 10, replace=False)
        # Summary
        try:
            topic['topic_summary'] = summarize_topic(headlines)
            print(topic['topic_summary'])
        except InvalidRequestError:
            topic['topic_summary'] = "Error Making Summary From OpenAI API"
            print("OpenAI API request failed.")
    else:
        headlines = topic['headlines']

In [ ]:
# turn topic titles and summaries into a dataframe
topic_titles_df = pd.DataFrame(topic_titles)
topic_titles_df = topic_titles_df[['topic', 'topic_summary']]
stories = stories.merge(topic_titles_df, on='topic', how='left')

In [ ]:
stories.to_csv('output/stories-with-nlp.csv', index=False)

# Collect Metadata

In [ ]:
# loop through topic_titles
topic_metadata = []
for topic in topic_titles:
    # grab topic, num_articles, and summary only
    topic = {k:v for k,v in topic.items() if k in ['topic', 'num_articles', 'topic_summary']}
    topic_metadata.append(topic)

topic_metadata

In [ ]:
# read output/metadata.json
import json
with open('output/metadata.json') as f:
    metadata = json.load(f)

metadata['topics'] = topic_metadata

# write metadata back to json file
with open('output/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=4)

metadata

In [ ]:
# collect top keywords
top_keywords = stories\
    .explode('keywords')\
    .groupby('keywords')\
    .size()\
    .reset_index(name='count')\
    .sort_values(by='count', ascending=False)\
    .head(100)
    